In [ ]:
import pandas as pd
import numpy as np
train_txt = '../../DataSets/oppo_data_ronud2_20181107/data_train.txt'
val_txt = '../../DataSets/oppo_data_ronud2_20181107/data_vali.txt'
train_data = pd.read_table(train_txt,
        names= ['prefix','query_prediction','title','tag','label'], header= None, encoding='utf-8',quoting=3).astype(str)
val_data = pd.read_table(val_txt,
        names = ['prefix','query_prediction','title','tag','label'], header = None, encoding='utf-8',quoting=3).astype(str)

data = pd.concat([train_data, val_data], ignore_index=True)

data['prefix'] = data['prefix'].apply(lambda x: x.lower())
data['title'] = data['title'].apply(lambda x: x.lower())

data['prefix_title'] = data['prefix'].apply(lambda x: x + "_")
data['prefix_title'] = data['prefix_title'] + data['title']

data['prefix_title_tag'] = data['prefix_title'].apply(lambda x: x + "_")
data['prefix_title_tag'] = data['prefix_title_tag'] + data['tag']

data['label'] = data['label'].apply(lambda x: int(x))

len_train = train_data.shape[0]
len_val = val_data.shape[0]

train_data = data[:len_train]
val_data = data[len_train:]

print("len_train",train_data.shape[0])
print("len_val",val_data.shape[0])

In [ ]:
from sklearn.metrics import classification_report,f1_score
prefixs = train_data.groupby(['prefix'], as_index=False)['prefix'].agg({'cnt': 'count'})['prefix'].tolist()
print('prefixs : ', len(prefixs))

# val model 1
val_feat_in = val_data[val_data['prefix'].isin(prefixs)].copy()
# val model2
val_feat_not_in = val_data[~val_data['prefix'].isin(prefixs)].copy()

val_feat = pd.concat([val_feat_in,val_feat_not_in])
val_feat['label'] = val_feat['label'].apply(lambda x : int(x))

len_in = val_feat_in.shape[0]
len_not_in = val_feat_not_in.shape[0]

print("len of val in prefixs : ", val_feat_in.shape[0])
print("len of val not in prefixs : ", val_feat_not_in.shape[0])

# pred = pd.read_table('pred_val_new_distance_tag_ctr_with_3_online_threshold_0.38_feat_0.05_200_0.3853734688370733_21285_0.4076336624065295_0.7538809855639979.csv',
#         names = ['predicted_score','label'],header=0 , encoding='utf-8',sep=',').astype(str)

pred1 = pd.read_table('../pred_val_0.36_0.37_lower_rm_prefix_id_online_threshold_0.38_feat_0.05_278_0.38538941082662853_22041.csv',
        names = ['predicted_score','label'],header = 0, encoding='utf-8',sep=',').astype(str)

pred2 = pd.read_table('../pred_val_0.36_0.37_lower_rm_prefix_id_online_threshold_0.38_feat_0.05_278_0.38538941082662853_22041.csv',
        names = ['predicted_score','label'],header = 0, encoding='utf-8',sep=',').astype(str)

#重新根据score预测label
pred_in = pred1[:len_in]
pred_not_in = pred2[len_in:]
pred_in['label'] = pred_in['predicted_score'].apply(lambda x: 1 if float(x) > 0.36 else 0)
pred_not_in['label'] = pred_not_in['predicted_score'].apply(lambda x: 1 if float(x) > 0.37 else 0)
pred = pd.concat([pred_in,pred_not_in])

val_feat['predicted_label'] = pred['label'].values
val_feat['predicted_score'] = pred['predicted_score'].values

val_feat['predicted_label'] = val_feat['predicted_label'].apply(lambda x:int(x))
val_feat['label'] = val_feat['label'].apply(lambda x:int(x))

print(f1_score(val_feat['label'],val_feat['predicted_label']))

prefixsAndCount = train_data.groupby(['prefix_title_tag'], as_index=False)['prefix_title_tag'].agg({'cnt': 'count'})
prefix_title_tag = prefixsAndCount['prefix_title_tag'].tolist()
print("len of prefix_title_tag:",len(prefix_title_tag))
d = {}
for i,row in prefixsAndCount.iterrows():
    d[row['prefix_title_tag']] = row['cnt']

In [ ]:
print(f1_score(val_feat['label'],val_feat['predicted_label']))

val_all_in = val_feat[val_feat['prefix_title_tag'].isin(prefix_title_tag)].copy()
val_all_not_in = val_feat[~val_feat['prefix_title_tag'].isin(prefix_title_tag)].copy()

print("len of val in prefix_title_tag : ", val_all_in.shape[0])
print("len of val not in prefix_title_tag : ", val_all_not_in.shape[0])

prefix_counts = []
count1 = 0
count2 = 0
for i,row in val_all_in.iterrows():
    prefix_counts.append(d[row['prefix_title_tag']])
    if d[row['prefix_title_tag']] == 1:
        count1 = count1 + 1
    if d[row['prefix_title_tag']] <= 2:
        count2 = count2 + 1
print(count1,count2)

val_all_in['prefix_count'] = prefix_counts

val_feat_in_one = val_all_in[(val_all_in['prefix_count'] ==1) & (val_all_in['prefix_count'] == 1)]
val_feat_in_not_one = val_all_in[(val_all_in['prefix_count'] != 1) | (val_all_in['prefix_count'] != 1)]
# val_feat_in_one = val_feat_in
print(val_feat_in_one.shape[0])
# print(val_feat_in_one.head(10))
val_feat = pd.concat([val_feat_in_one,val_feat_in_not_one,val_all_not_in])
labels = val_feat['label'].values

print(f1_score(val_feat['label'],val_feat['predicted_label']))
print(f1_score(val_all_in['label'],val_all_in['predicted_label']))
print(f1_score(val_feat_in_one['label'],val_feat_in_one['predicted_label']))
print(val_feat_in_one[val_feat_in_one['label'] == val_feat_in_one['predicted_label']].shape)

In [ ]:
print(val_feat_in_one[val_feat_in_one['label'] == val_feat_in_one['predicted_label']].shape)

In [ ]:
prefix_title_tag_one = val_feat_in_one['prefix_title_tag'].values
print(len(prefix_title_tag_one))
train_data_one = train_data[train_data['prefix_title_tag'].isin(prefix_title_tag_one)]
print(train_data_one.shape)

train_data_one['label'] = train_data_one['label'].apply(lambda x : int(x))
temp = train_data_one.groupby('prefix_title_tag', as_index=False)['label'].agg({'click': 'sum', 'count': 'count'})
temp['ctr'] = temp['click'] / (temp['count'])
train_data_one = pd.merge(train_data_one, temp, on='prefix_title_tag', how='left')

train_label_dict = {}
for i,row in train_data_one.iterrows():
    if row['ctr'] > 0.38:
        train_label_dict[row['prefix_title_tag']] = 1
    else:
        train_label_dict[row['prefix_title_tag']] = 0

train_label = []
label = []
cnt = 0
for i,row in val_feat.iterrows():
    if row['prefix_title_tag'] in prefix_title_tag_one:
        train_label.append(train_label_dict[row['prefix_title_tag']])
        label.append(train_label_dict[row['prefix_title_tag']])
        cnt = cnt + 1
    else:
        train_label.append(row['predicted_label'])

print(f1_score(val_feat_in_one['label'],label))
print(f1_score(labels,train_label))

# max_f1 = 0
# for ctr in np.arange(0.00, 1.00, 0.01):
#     train_label = []
#     train_label_dict = {}
#     for i,row in train_data_one.iterrows():
#         if row['ctr'] > ctr:
#             train_label_dict[row['prefix_title_tag']] = 1
#         else:
#             train_label_dict[row['prefix_title_tag']] = 0
#     for i,row in val_feat_in_one.iterrows():
#         train_label.append(train_label_dict[row['prefix_title_tag']])
#     #[val_feat_in_one,val_feat_in_not_one,val_all_not_in]
#     tmp_f1 = f1_score(labels,np.concatenate([train_label,val_feat_in_not_one['predicted_label'].values,val_all_not_in['predicted_label'].values]))
#     if tmp_f1 > max_f1:
# #         val_feat_in_one['predicted_label'] = train_label
#         max_f1 = tmp_f1
#         total_f1 = f1_score(val_feat_in_one['label'],train_label)
#         print('ctr:' , ctr , tmp_f1 , total_f1 , np.sum(val_feat_in_one['label'] == train_label))
    
# train_label = []
# train_label_dict = {}
# for i,row in train_data_one.iterrows():
#     train_label_dict[row['prefix_title_tag']] = int(row['label'])
    
# for i,row in val_feat_in_one.iterrows():
#     train_label.append(train_label_dict[row['prefix_title_tag']])

# print(f1_score(val_feat_in_one['label'],train_label))

In [ ]:
val_feat['predicted_label'] = train_label
print(f1_score(val_feat['label'],val_feat['predicted_label']))
print(np.sum(val_feat_in_one['label'] == label))